In [1]:
!pip install keras
!pip install tensorflow
!pip install tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
import tensorflow as tf

# Load EfficientNetV2B0 pretrained on ImageNet
model = tf.keras.applications.EfficientNetV2B0(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=True
)


29403144/29403144 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


##Baseline to TFLite(no quantization)

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("efficientnetv2_b0_baseline.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp3zs8u9_c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  136603066432976: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066432784: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066433360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066432016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066431824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063452880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454032: TensorSpec(shape=(), dtype=tf.resource, name=N

In [8]:
# Compare model sizes

print(f"Model Sizes (KB): efficientnetv2_b0_baseline.tflite: {os.path.getsize("/content/efficientnetv2_b0_baseline.tflite") / 1024:.2f} KB")

Model Sizes (KB): efficientnetv2_b0_baseline.tflite: 27859.49 KB


##Dynamic Range Quantization

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_dynamic = converter.convert()
with open("efficientnetv2_b0_dynamic.tflite", "wb") as f:
    f.write(tflite_model_dynamic)

Saved artifact at '/tmp/tmpvu3fmj9m'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  136603066432976: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066432784: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066433360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066432016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066431824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063452880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454032: TensorSpec(shape=(), dtype=tf.resource, name=N

In [10]:
# Compare model sizes

print(f"Model Sizes (KB): efficientnetv2_b0_dynamic.tflite: {os.path.getsize("/content/efficientnetv2_b0_dynamic.tflite") / 1024:.2f} KB")

Model Sizes (KB): efficientnetv2_b0_dynamic.tflite: 7692.05 KB


##Float16 Quantization

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model_fp16 = converter.convert()
with open("efficientnetv2_b0_fp16.tflite", "wb") as f:
    f.write(tflite_model_fp16)


Saved artifact at '/tmp/tmpm5nt8y57'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  136603066432976: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066432784: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136603066433360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063453264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066432016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603066431824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063452880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136603063454032: TensorSpec(shape=(), dtype=tf.resource, name=N

In [12]:
print(f"Model Sizes (KB): efficientnetv2_b0_fp16.tflite: {os.path.getsize("/content/efficientnetv2_b0_fp16.tflite") / 1024:.2f} KB")

Model Sizes (KB): efficientnetv2_b0_fp16.tflite: 13995.01 KB


##Full Integer Quantization (EdgeTPU compatible)

In [ ]:
import numpy as np

def representative_data_gen():
    for _ in range(100):
        # Replace this with real ImageNet images if possible
        img = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
        img = tf.keras.applications.efficientnet_v2.preprocess_input(img.astype(np.float32))
        img = np.expand_dims(img, axis=0)  # add batch dim
        yield [img]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Force int8 everywhere
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_int8 = converter.convert()
with open("efficientnetv2_b0_int8.tflite", "wb") as f:
    f.write(tflite_model_int8)


In [ ]:
import os
import numpy as np
import tensorflow as tf  # Ensure TensorFlow is imported

# Compare model sizes
model_files = [
    "model_baseline.tflite",
    "model_integer_quant.tflite",
    "model_dynamic_quant.tflite",
    "model_float16_quant.tflite"
]

print("Model Sizes (KB):")
for file in model_files:
    print(f"{file}: {os.path.getsize(file) / 1024:.2f} KB")

# Evaluate accuracy of a TFLite model on the test set
def evaluate_tflite_model(tflite_model_path):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_dtype = input_details[0]['dtype']
    input_scale, input_zero_point = input_details[0]['quantization']

    correct_predictions = 0

    for i in range(len(x_test)):
        # Prepare one input sample (shape: [1, 28, 28, 1])
        input_data = x_test[i:i+1].astype("float32")

        # Quantize the input if model expects int8
        if input_dtype == np.int8:
            input_data = input_data / input_scale + input_zero_point
            input_data = np.round(input_data).astype(np.int8)
        elif input_dtype == np.float16:
            input_data = input_data.astype(np.float16)

        # Set the tensor and invoke the interpreter
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        # Get prediction and compare to true label
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_label = np.argmax(output_data, axis=1)
        true_label = np.argmax(y_test[i:i+1], axis=1)

        if predicted_label == true_label:
            correct_predictions += 1

    # Return classification accuracy
    return correct_predictions / len(x_test)

# Display accuracy of each model
print("\nModel Accuracies:")
print(f"Baseline: {original_accuracy:.4f}")
print(f"Integer Quantization: {evaluate_tflite_model('model_integer_quant.tflite'):.4f}")
print(f"Dynamic Range Quantization: {evaluate_tflite_model('model_dynamic_quant.tflite'):.4f}")
print(f"Float 16 Quantization: {evaluate_tflite_model('model_float16_quant.tflite'):.4f}")


In [ ]:
# Verify the model type
model = create_cnn_model()
print(f"Model type: {type(model)}")

from tensorflow_model_optimization.quantization.keras import quantize_model

# Prepare the quantization-aware model
qat_model = quantize_model(create_cnn_model())

# Compile the QAT model
qat_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the QAT model
print("Training the QAT Model...")
qat_history = qat_model.fit(x_train, y_train, epochs=2, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the QAT-trained model
qat_accuracy = qat_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"QAT Model Accuracy: {qat_accuracy:.4f}")
